In [1]:
from pathlib import Path
from typing import Annotated, Union

import typer
from peft import AutoPeftModelForCausalLM, PeftModelForCausalLM
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    PreTrainedModel,
    PreTrainedTokenizer,
    PreTrainedTokenizerFast,
)

/home/reymond/anaconda3/envs/afac2024/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ModelType = Union[PreTrainedModel, PeftModelForCausalLM]
TokenizerType = Union[PreTrainedTokenizer, PreTrainedTokenizerFast]

In [3]:
def _resolve_path(path: Union[str, Path]) -> Path:
    return Path(path).expanduser().resolve()


def load_model_and_tokenizer(model_dir: Union[str, Path]) -> tuple[ModelType, TokenizerType]:
    model_dir = _resolve_path(model_dir)
    if (model_dir / 'adapter_config.json').exists():
        model = AutoPeftModelForCausalLM.from_pretrained(
            model_dir, trust_remote_code=True, device_map='auto'
        )
        tokenizer_dir = model.peft_config['default'].base_model_name_or_path
    else:
        model = AutoModelForCausalLM.from_pretrained(
            model_dir, trust_remote_code=True, device_map='auto'
        )
        tokenizer_dir = model_dir
    tokenizer = AutoTokenizer.from_pretrained(
        tokenizer_dir, trust_remote_code=True
    )
    return model, tokenizer

In [4]:
model_dir = "output/checkpoint-3000"

model, tokenizer = load_model_and_tokenizer(model_dir)

Loading checkpoint shards: 100%|██████████| 3/3 [03:02<00:00, 60.77s/it]
Setting eos_token is not supported, use the default one.
Setting pad_token is not supported, use the default one.
Setting unk_token is not supported, use the default one.


In [5]:
test_input = "你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。query是：我打算用100万元买三羊马的股票，如果按照三羊马的最高价来计算，我能买多少股呢？ \n query中提到的产品标准名可能是：富国中证1000优选股票型证券投资基金C类、富国中证1000优选股票型证券投资基金A类、前海开源股息率100强等权重股票型证券投资基金、前海开源强势共识100强等权重股票型证券投资基金、国泰君安中证1000优选股票型发起式证券投资基金C类、国泰君安中证1000优选股票型发起式证券投资基金A类、三羊马、万马股份、万家元贞量化选股股票型证券投资基金C类、万家元贞量化选股股票型证券投资基金A类、富国沪深300基本面精选股票型证券投资基金C类、富国沪深300基本面精选股票型证券投资基金A类、南方中证500量化增强股票型发起式证券投资基金C类、南方中证500量化增强股票型发起式证券投资基金A类、富国中证500基本面精选股票型发起式证券投资基金C类、富国中证500基本面精选股票型发起式证券投资基金A类、银河定投宝中证腾讯济安价值100A股指数型发起式证券投资基金、华夏中证智选1000价值稳健策略交易型开放式指数证券投资基金、长盛中证100指数证券投资基金、融通深证100指数证券投资基金、广发多元新兴股票型证券投资基金、华富中证100指数证券投资基金、万家消费成长股票型证券投资基金、银华中国梦30股票型证券投资基金、南方天元新产业股票型证券投资基金、东吴双三角股票型证券投资基金C类、东吴双三角股票型证券投资基金A类、长城消费30股票型证券投资基金C类、长城消费30股票型证券投资基金A类、诺安中证100指数证券投资基金C类、诺安中证100指数证券投资基金A类、融通深证100指数证券投资基金C类、招商深证100指数证券投资基金C类、招商深证100指数证券投资基金A类、工银瑞信聚焦30股票型证券投资基金、富国消费精选30股票型证券投资基金、富国医药成长30股票型证券投资基金、国泰纳斯达克100指数证券投资基金、南方中证100指数证券投资基金C类、南方中证100指数证券投资基金A类、华宝兴业中证100指数证券投资基金、中金精选股票型集合资产管理计划C类、中金精选股票型集合资产管理计划A类、中金新锐股票型集合资产管理计划C类、中金新锐股票型集合资产管理计划A类、中小企业100交易型开放式指数基金、招商沪深300高贝塔指数证券投资基金、建信深证100指数增强型证券投资基金、工银瑞信工业4.0股票型证券投资基金、博时工业4.0主题股票型证券投资基金"
response, _ = model.chat(tokenizer, test_input)
print(response)

Both `max_new_tokens` (=1024) and `max_length`(=8192) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


非常抱歉，我无法通过编排API来得到你的问题答案，因为API需要具体的请求参数。你的问题中提到的"三羊马"可能是指某个特定的股票，但是API需要知道这个股票的具体代码或者名称，我才能帮你计算出你可以购买的股票数量。如果你能提供更多信息，我会很乐意帮助你。
